In [1]:
import pandas as pd  # таблицы
import requests as rq  # запросы сети
import re # регулярные выражения
from bs4 import BeautifulSoup  # парсинг
from fake_useragent import UserAgent  # эмуляция браузера

# обработка временной зоны
import pytz
import datetime

dir = "D:\HSE\Pump_Dump\TG\\"  # папка проекта

In [2]:
# получение "сырых" данных с диска по каналам
def channel_raw_data(channel):
    # формируем путь к файлу с данными канала
    channel_path = dir + channel + ".csv"
    # считываем существующие данные, если канал есть, если нет создаем файл
    try:
        # загрузка файла
        df = pd.read_csv(channel_path, encoding="utf-8", sep="|", keep_default_na=False)
    except:
        cols = {
            "CHANNEL": [],
            "POST": [],
            "NEWS_DATE": [],
            "NEWS_TIME": [],
            "NEWS_TEXT": [],
        }
        df = pd.DataFrame(cols)
        df.to_csv(
            channel_path,
            index=False,
            encoding="utf-8",
            sep="|",
        )
    return df

In [3]:
# описание канала
def channel_info(channel):
    # главная страница канала
    page = rq.get(
        "https://t.me/s/" + channel, headers={"User-Agent": UserAgent().chrome}
    )
    raw_text = BeautifulSoup(page.text.replace("<br/>", " "), features="html.parser")
    # получаем номер крайней новости и количество подписчиков
    to_number = int(str(raw_text.find_all("link")[1].get("href")).split("=")[1])
    channel_counter = raw_text.find(
        "div", attrs={"class": "tgme_header_counter"}
    ).text.split(" ")[0]
    return (to_number, channel_counter)

In [4]:
# загрузка конкретной новости
def load_post(channel, post):
    # переменные для загрузки
    news_date = ""
    news_time = ""
    news_text = ""
    try:
        # обращаемся к странице
        page_link = "https://t.me/" + channel + "/" + str(post) + "?embed=1"
        page = rq.get(page_link)
        # загружаем ее в парсер и сразу удаляем переносы строки, которые встречаются в тексте страницы
        raw_text = BeautifulSoup(page.text.replace("<br/>", " "), features="html.parser")
        # дата и время новости c учетом временной зоны # Europe / Moscow (+3)
        news_date = raw_text.find("time").attrs["datetime"]
        news_date = datetime.datetime.strptime(news_date, "%Y-%m-%dT%H:%M:%S%z")
        tz = pytz.timezone("Etc/GMT-3")
        news_date = tz.normalize(news_date.astimezone(tz))
        news_time = news_date.strftime("%H:%M")
        news_date = news_date.strftime("%d-%m-%Y")
        # текст новости
        news_text = raw_text.find(
            "div", attrs={"class": "tgme_widget_message_text js-message_text"}
        ).text
        # оставляем только необходимые знаки
        news_text = re.sub(r"[^0-9А-Яа-яA-Za-zёЁ-–—,.%«»:/?!$№€₽ ]", " ", news_text)
        news_text = " ".join(news_text.split())
    except:
        # если ошибка при обработке, в текст пишем об ошибке
        news_text = "error"
    data = [
        {
            "CHANNEL": channel,
            "POST": int(post),
            "NEWS_DATE": news_date,
            "NEWS_TIME": news_time,
            "NEWS_TEXT": news_text,
        }
    ]
    if news_text=="error":data="error"
    return data

In [5]:
channels = pd.read_csv(dir + "!channels.csv", delimiter="|")
# обрабатываем каждый канал
for channel in channels["CHANNEL"].values:
    # загружаем данные из корпуса
    channel_df = channel_raw_data(channel)
    # получаем номер крайней новости в корпусе
    from_date = channel_df["POST"].max()
    # загружаем данные о канале
    channel_data = channel_info(channel)
    # дата крайней новости на канале
    to_date = channel_data[0]
    # актуальное количество подписчиков канала
    channel_counter = channel_data[1]
    # обновляем количество подписчиков в списке каналов (RELIABILITY)
    channels.loc[channels["CHANNEL"] == channel, "RELIABILITY"] = channel_counter
    # и обновляем его в списке
    channels.to_csv(
        dir + "!channels.csv",
        index=False,
        encoding="utf-8",
        sep="|",
    )
    # если новый / пустой файл, загружаем крайние N записей
    try:
        int(from_date)
    except:
        from_date = int(to_date) - 1000
    # общая информация о предстоящей загрузке
    print(
        "Канал: ",
        channel,
        "Подписчиков: ",
        channel_counter,
        "Первая новость в корпусе: ",
        channel_df["POST"].min(),
        "Крайняя новость в корпусе: ",
        channel_df["POST"].max(),
        "Крайняя новость в ТГ: ",
        to_date,
        "Будет загружено",
        int(to_date) - int(from_date) - 1,
        sep="\n",
    )

    # в цикле проходим по всем незагруженным новостям
    # для этого создаем временный фрейм
    temp_frame = pd.DataFrame()
    #from_date = 16500  ### для разовых загрузок
    #to_date = 20895  ### для разовых загрузок
    for i in range(from_date + 1, to_date):
        # получаем словарь из данных страницы
        temp_row = load_post(channel, i)
        print(temp_row)
        # если нет ошибочной записи, то добавляем ее во временный фрейм
        if temp_row != "error":
            temp_frame = pd.concat(
                [temp_frame, pd.DataFrame.from_dict(temp_row)], ignore_index=True
            )
    # в некоторых новостях встречаются дубли текстов, удаляем их
    temp_frame.drop_duplicates(subset="NEWS_TEXT", keep="first", inplace=True)
    # добавляем новости в основной фрейм канала
    channel_df = pd.concat([channel_df, temp_frame], ignore_index=False)
    # POST должен быть числовым
    channel_df["POST"] = channel_df["POST"].astype(int)
    # перед обновлением файла сортируем по убыванию даты для удобства просмотра файла
    channel_df = channel_df.sort_values(by="POST", ascending=False)
    channel_df.to_csv(
        dir + channel + ".csv",
        index=False,
        encoding="utf-8",
        sep="|",
    )

Канал: 
signals_moex
Подписчиков: 
12.7K
Первая новость в корпусе: 
4401
Крайняя новость в корпусе: 
6067
Крайняя новость в ТГ: 
6072
Будет загружено
4
[{'CHANNEL': 'signals_moex', 'POST': 6068, 'NEWS_DATE': '07-02-2024', 'NEWS_TIME': '11:58', 'NEWS_TEXT': 'RASP крупная заявка на покупку 2к лотов 8 528 000 руб по цене 426.40 До плотности 11 пунктов 0,129% Устанавливайте бесплатный терминал для скальпинга CScalp и следите за плотностями в режиме реального времени. Как отрабатывать такие ситуации смотрите здесь'}]
[{'CHANNEL': 'signals_moex', 'POST': 6069, 'NEWS_DATE': '07-02-2024', 'NEWS_TIME': '12:07', 'NEWS_TEXT': 'SHORT SMLT from 3 733.5 р. stop loss 3 741 р. 30m TF. Бумага находится в нисходящем тренде и сформировала наклонку из каскада уровней, жду подход к уровню 3733,5 и буду заходить в пробой, цель собрать каскад, спот исходя из вашего РМ. Бесплатный терминал для скальпинга здесь. Открыть график ПробойНаклонки'}]
[{'CHANNEL': 'signals_moex', 'POST': 6070, 'NEWS_DATE': '07-02-202